#### Elliptic Curve Cryptography example code

example taken from:

https://jeremykun.com/2014/02/24/elliptic-curves-as-python-objects/

https://github.com/j2kun/elliptic-curves-rationals

need to run the code at the bottom 1st

other links:

https://arstechnica.com/information-technology/2013/10/a-relatively-easy-to-understand-primer-on-elliptic-curve-cryptography/

In [75]:
EllipticCurve(a=17, b=1)
#y^2 = x^3 + 17x + 1

y^2 = x^3 + 17x + 1

In [76]:
C = EllipticCurve(a=frac(-2), b=frac(4))
P = Point(C, frac(3), frac(5))
Q = Point(C, frac(-2), frac(0))
zero = Ideal(C)

In [79]:
P + Q + P + P + P

(Fraction(23518, 6241), Fraction(3485400, 493039))

In [0]:
# examples
C = EllipticCurve(a=frac(-2), b=frac(4))
P = Point(C, frac(3), frac(5))
Q = Point(C, frac(-2), frac(0))
zero = Ideal(C)

P + Q
Q + P
5*P
Q - 3*P

Example of Elliptic Curve Diffie Hellman [youtube](https://www.youtube.com/watch?v=F3zzNa42-tQ&t=922s)

I changed some of the numbers because the math is not being done as non-modulo and the points didn't lie on the curve.

In [30]:
a = -2
b = 4
#p = 17
#n = 19
#h = 1
C = EllipticCurve(a, b)
# G = 5,1
G = Point(C, frac(3), frac(5))
C

y^2 = x^3 + -2x + 4

In [38]:
# Bob's secret number = 9
B = 2*G
B  # Bob sends this number to Alice

(Fraction(1, 4), Fraction(15, 8))

In [39]:
# Alice's secret number = 3
A = 3*G
A  # Alice sends this number to Bob

(Fraction(-237, 121), Fraction(845, 1331))

In [41]:
# Bob computes
K = 2*A
K

(Fraction(829495681, 13823524), Fraction(-23883862896945, 51395862232))

In [42]:
# Alice computes
K1 = 3*B
K1

(Fraction(829495681, 13823524), Fraction(-23883862896945, 51395862232))

In [43]:
K - K1

Ideal

Using the functions below, try the key exchange example from [Youtube](https://www.youtube.com/watch?v=F3zzNa42-tQ&t=922s)

In [23]:
#p = 17
#n = 19
#h = 1
# G = 5,1

Pcurve = 17 # The proven prime
N = 19 # Number of points in the field
Acurve = 2
Bcurve = 2 # These two defines the elliptic curve. y^2 = x^3 + Acurve * x + Bcurve
Gx = 5
Gy = 1
GPoint = (Gx,Gy) # This is our generator point.

In [35]:
#Individual Transaction/Personal Information
Bobs_privKey = 9
Bobs_PublicKey = EccMultiply(GPoint,Bobs_privKey)
print("Bob's private key:", Bobs_privKey)
print("Bob's public key:", Bobs_PublicKey)

Bob's private key: 9
Bobs public key: (7, 6)


In [36]:
#Individual Transaction/Personal Information
Alices_privKey = 3
Alices_PublicKey = EccMultiply(GPoint,Alices_privKey)
print("Alice's private key:", Alices_privKey)
print("Alice's public key:", Alices_PublicKey)

Alice's private key: 3
Alice's public key: (10, 6)


In [33]:
K_Bob = EccMultiply(Alices_PublicKey,Bobs_privKey)
K_Bob

(13, 7)

In [34]:
K_Alice = EccMultiply(Bobs_PublicKey,Alices_privKey)
K_Alice

(13, 7)

In [1]:
from fractions import Fraction as frac

In [2]:
class EllipticCurve(object):
   def __init__(self, a, b):
      # assume we're already in the Weierstrass form
      self.a = a
      self.b = b

      self.discriminant = -16 * (4 * a*a*a + 27 * b * b)
      if not self.isSmooth():
         raise Exception("The curve %s is not smooth!" % self)


   def isSmooth(self):
      return self.discriminant != 0


   def testPoint(self, x, y):
      return y*y == x*x*x + self.a * x + self.b


   def __str__(self):
      return 'y^2 = x^3 + %sx + %s' % (self.a, self.b)


   def __repr__(self):
      return str(self)


   def __eq__(self, other):
      return (self.a, self.b) == (other.a, other.b)

class Point(object):
   def __init__(self, curve, x, y):
      self.curve = curve # the curve containing this point
      self.x = x
      self.y = y

      if not curve.testPoint(x,y):
         raise Exception("The point %s is not on the given curve %s!" % (self, curve))


   def __str__(self):
      return "(%r, %r)" % (self.x, self.y)


   def __repr__(self):
      return str(self)


   def __neg__(self):
      return Point(self.curve, self.x, -self.y)


   def __add__(self, Q):
      if self.curve != Q.curve:
         raise Exception("Can't add points on different curves!")
      if isinstance(Q, Ideal):
         return self

      x_1, y_1, x_2, y_2 = self.x, self.y, Q.x, Q.y

      if (x_1, y_1) == (x_2, y_2):
         if y_1 == 0:
            return Ideal(self.curve)

         # slope of the tangent line
         m = (3 * x_1 * x_1 + self.curve.a) / (2 * y_1)
      else:
         if x_1 == x_2:
            return Ideal(self.curve)

         # slope of the secant line
         m = (y_2 - y_1) / (x_2 - x_1)

      x_3 = m*m - x_2 - x_1
      y_3 = m*(x_3 - x_1) + y_1

      return Point(self.curve, x_3, -y_3)


   def __sub__(self, Q):
      return self + -Q

   def __mul__(self, n):
      if not isinstance(n, int):
         raise Exception("Can't scale a point by something which isn't an int!")

      if n < 0:
         return -self * -n

      if n == 0:
         return Ideal(self.curve)

      Q = self
      R = self if n & 1 == 1 else Ideal(self.curve)

      i = 2
      while i <= n:
         Q += Q
         if n & i == i:
             R += Q
         i = i << 1
      return R


   def __rmul__(self, n):
      return self * n

   def __list__(self):
      return [self.x, self.y]

   def __eq__(self, other):
      if type(other) is Ideal:
         return False

      return self.x, self.y == other.x, other.y

   def __ne__(self, other):
      return not self == other

   def __getitem__(self, index):
      return [self.x, self.y][index]


class Ideal(Point):
   def __init__(self, curve):
      self.curve = curve

   def __neg__(self):
      return self

   def __str__(self):
      return "Ideal"

   def __add__(self, Q):
      if self.curve != Q.curve:
         raise Exception("Can't add points on different curves!")
      return Q

   def __mul__(self, n):
      if not isinstance(n, int):
         raise Exception("Can't scale a point by something which isn't an int!")
      else:
         return self

   def __eq__(self, other):
      return type(other) is Ideal

### plotting the graph of an elliptic curve

In [12]:
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# find the roots of the LHS
#(1)x**3 + (0)x**2 +(-a)x + (b)
a = 5 #1.85
b = 5
coeff = [1, 0, -a, b]
r = np.roots(coeff)
r[0].real

-2.6273650847118302

In [18]:
# set the lower limit to a smidge above the root so the sqrt is positive
X = np.arange(r[0].real+0.0001, 4,.01)
Y = np.sqrt(X**3 - a*X + b)

In [19]:
plt.plot(X, Y, 'blue')
plt.plot(X, -Y, 'blue')
plt.show()

##### examples in Evan Dummit's paper
file:///home/chronos/u-1923d13e3a0d20adb4744c19a86a3f50eec4356b/Downloads/cryptography_5_elliptic_curves_in_cryptography.pdf

###### 5.1.2 The Addition Law, example
Given the points $P_{1} = (1, 2)$ and $P_{2} = (3, 4)$ on the elliptic curve $y^{2} = x^{3} − 7x + 10$, find the sums $P_{1} + P_{2}$ and $(P_{1} + P_{2}) + P_{2}$.

In [3]:
C = EllipticCurve(a=frac(-7), b=frac(10))
C

y^2 = x^3 + -7x + 10

In [7]:
P1 = Point(C, frac(1), frac(2))
P2 = Point(C, frac(3), frac(4))
zero = Ideal(C)

In [8]:
P1+P2 # ans is (-3,2)

(Fraction(-3, 1), Fraction(2, 1))

In [9]:
(P1 + P2) + P2 #ans (1/9, -82/27)

(Fraction(1, 9), Fraction(-82, 27))

In [10]:
P2 + P2

(Fraction(1, 4), Fraction(23, 8))

#### write some code to find valid X,Y pairs

#### random primes

In [3]:
import random
import math

In [11]:
def primeNr(interval):
    print("Prime numbers from 2 to ", interval)

    for i in range(2, interval + 1):
        for j in range(2, int(math.sqrt(i)) + 1):
            if i % j == 0:
                break
        else:
            print(i)

primeNr(20)

Prime numbers from 2 to  20
2
3
5
7
11
13
17
19


## example code from youtube

https://www.youtube.com/watch?v=wpLQZhqdPaA

https://github.com/wobine/blackboard101/blob/master/EllipticCurvesPart4-PrivateKeyToPublicKey.py

http://www.secg.org/

note: the math is not working for: EccMultiply(GPoint,privKey), needed to fix up int divide, added // to modin() function

In [62]:
print()
print( "******* Public Key Generation *********") 
print()
PublicKey = EccMultiply(GPoint,privKey)
print("the private key:")
print(privKey)
print()
print("the uncompressed public key (not address):")
print(PublicKey)
print()
print("the uncompressed public key (HEX):")
#print("04" + "%064x" % PublicKey[0] + "%064x" % PublicKey[1])
print('04 {:.1f} {:.1f}'.format(PublicKey[0], PublicKey[1]))
print()
print("the official Public Key - compressed:")
if PublicKey[1] % 2 == 1: # If the Y value for the Public Key is odd.
    #print("03"+str(hex(PublicKey[0])[2:-1]).zfill(64))
    print("03"+str(hex(int(PublicKey[0]))[2:-1]).zfill(64))
else: # Or else, if the Y value is even.
    #print("02"+str(hex(PublicKey[0])[2:-1]).zfill(64))
    print("02"+str(hex(int(PublicKey[0]))[2:-1]).zfill(64))


******* Public Key Generation *********

the private key:
72759466100064397073952777052424474334519735946222029294952053344302920927294

the uncompressed public key (not address):
(3423904187495496827825042940737875085827330420143621346629173781207857376010, 75711134420273723792089656449854389054866833762486990555172221523628676983696)

the uncompressed public key (HEX):
04 3423904187495496695191934108453614092249892327363168260152493940687224438784.0 75711134420273721024244848487950043951522146904316331060657446517288512520192.0

the official Public Key - compressed:
0200791dc70b75aa995213244ad3f4886d74d61ccd3ef658243fcad14c9ccee2b0


+

In [61]:
# Super simple Elliptic Curve Presentation. No imported libraries, wrappers, nothing. 
# For educational purposes only. Remember to use Python 2.7.6 or lower. You'll need to make changes for Python 3.

# Below are the public specs for Bitcoin's curve - the secp256k1

Pcurve = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1 # The proven prime
N=0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141 # Number of points in the field
Acurve = 0; Bcurve = 7 # These two defines the elliptic curve. y^2 = x^3 + Acurve * x + Bcurve
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
GPoint = (Gx,Gy) # This is our generator point. Trillions of dif ones possible

#Individual Transaction/Personal Information
privKey = 0xA0DC65FFCA799873CBEA0AC274015B9526505DAAAED385155425F7337704883E #replace with any private key


In [4]:
# Super simple Elliptic Curve Presentation. No imported libraries, wrappers, nothing. 
# For educational purposes only. Remember to use Python 2.7.6 or lower. You'll need to make changes for Python 3.

def modinv(a,n=Pcurve): #Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1,0
    low, high = a%n,n
    while low > 1:
        ratio = high//low     # added double slash for int divide //
        nm, new = hm-lm*ratio, high-low*ratio
        lm, low, hm, high = nm, new, lm, low
    return lm % n

def ECadd(a,b): # Not true addition, invented for EC. Could have been called anything.
    LamAdd = ((b[1]-a[1]) * modinv(b[0]-a[0],Pcurve)) % Pcurve
    x = (LamAdd*LamAdd-a[0]-b[0]) % Pcurve
    y = (LamAdd*(a[0]-x)-a[1]) % Pcurve
    return (x,y)

def ECdouble(a): # This is called point doubling, also invented for EC.
    Lam = ((3*a[0]*a[0]+Acurve) * modinv((2*a[1]),Pcurve)) % Pcurve
    x = (Lam*Lam-2*a[0]) % Pcurve
    y = (Lam*(a[0]-x)-a[1]) % Pcurve
    return (x,y)

def EccMultiply(GenPoint,ScalarHex): #Double & add. Not true multiplication
    if ScalarHex == 0 or ScalarHex >= N: raise Exception("Invalid Scalar/Private Key")
    ScalarBin = str(bin(ScalarHex))[2:]
    Q=GenPoint
    for i in range (1, len(ScalarBin)): # This is invented EC multiplication.
        Q=ECdouble(Q); # print "DUB", Q[0]; print
        if ScalarBin[i] == "1":
            Q=ECadd(Q,GenPoint); # print "ADD", Q[0]; print
    return (Q)

## A block of code from part 5:

https://github.com/wobine/blackboard101/blob/master/EllipticCurvesPart5-TheMagic-SigningAndVerifying.py

In [0]:
# Python 2.7.6 - Super simple Elliptic Curve Presentation. No imported libraries, wrappers, nothing. # For educational purposes only
# Below are the public specs for Bitcoin's curve - the secp256k1
Pcurve = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1 # The proven prime
N=0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141 # Number of points in the field
Acurve = 0; Bcurve = 7 # This defines the curve. y^2 = x^3 + Acurve * x + Bcurve
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
GPoint = (Gx,Gy) # This is our generator point. Tillions of dif ones possible

#Individual Transaction/Personal Information
privKey = 75263518707598184987916378021939673586055614731957507592904438851787542395619 #replace with any private key
RandNum = 28695618543805844332113829720373285210420739438570883203839696518176414791234 #replace with a truly random number
HashOfThingToSign = 86032112319101611046176971828093669637772856272773459297323797145286374828050 # the hash of your message/transaction

def modinv(a,n=Pcurve): #Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1,0
    low, high = a%n,n
    while low > 1:
        ratio = high/low
        nm, new = hm-lm*ratio, high-low*ratio
        lm, low, hm, high = nm, new, lm, low
    return lm % n

def ECadd(xp,yp,xq,yq): # Not true addition, invented for EC. It adds Point-P with Point-Q.
    m = ((yq-yp) * modinv(xq-xp,Pcurve)) % Pcurve
    xr = (m*m-xp-xq) % Pcurve
    yr = (m*(xp-xr)-yp) % Pcurve
    return (xr,yr)

def ECdouble(xp,yp): # EC point doubling,  invented for EC. It doubles Point-P.
    LamNumer = 3*xp*xp+Acurve
    LamDenom = 2*yp
    Lam = (LamNumer * modinv(LamDenom,Pcurve)) % Pcurve
    xr = (Lam*Lam-2*xp) % Pcurve
    yr = (Lam*(xp-xr)-yp) % Pcurve
    return (xr,yr)

def EccMultiply(xs,ys,Scalar): # Double & add. EC Multiplication, Not true multiplication
    if Scalar == 0 or Scalar >= N: raise Exception("Invalid Scalar/Private Key")
    ScalarBin = str(bin(Scalar))[2:]
    Qx,Qy=xs,ys
    for i in range (1, len(ScalarBin)): # This is invented EC multiplication.
        Qx,Qy=ECdouble(Qx,Qy); # print "DUB", Qx; print
        if ScalarBin[i] == "1":
            Qx,Qy=ECadd(Qx,Qy,xs,ys); # print "ADD", Qx; print
    return (Qx,Qy)
'''
print; print "******* Public Key Generation *********"
xPublicKey, yPublicKey = EccMultiply(Gx,Gy,privKey)
print "the private key (in base 10 format):"; print privKey; print
print "the uncompressed public key (starts with '04' & is not the public address):"; print "04",xPublicKey,yPublicKey

print; print "******* Signature Generation *********"
xRandSignPoint, yRandSignPoint = EccMultiply(Gx,Gy,RandNum)
r = xRandSignPoint % N; print "r =", r
s = ((HashOfThingToSign + r*privKey)*(modinv(RandNum,N))) % N; print "s =", s

print; print "******* Signature Verification *********>>"
w = modinv(s,N)
xu1, yu1 = EccMultiply(Gx,Gy,(HashOfThingToSign * w)%N)
xu2, yu2 = EccMultiply(xPublicKey,yPublicKey,(r*w)%N)
x,y = ECadd(xu1,yu1,xu2,yu2)
print r==x; print
'''